# How to run a semantic skills from file
Now that you're familiar with Kernel basics, let's see how the kernel allows you to run Semantic Skills and Semantic Functions stored on disk. 

A Semantic Skill is a collection of Semantic Functions, where each function is defined with natural language that can be provided with a text file. 

Refer to our [glossary](https://github.com/microsoft/semantic-kernel/blob/main/docs/GLOSSARY.md) for an in-depth guide to the terms.

The repository includes some examples under the [samples](https://github.com/microsoft/semantic-kernel/tree/main/samples) folder.

For instance, [this](../../skills/FunSkill/Joke/skprompt.txt) is the **Joke function** part of the **FunSkill skill**:

```
WRITE EXACTLY ONE JOKE or HUMOROUS STORY ABOUT THE TOPIC BELOW.
JOKE MUST BE:
- G RATED
- WORKPLACE/FAMILY SAFE
NO SEXISM, RACISM OR OTHER BIAS/BIGOTRY.
BE CREATIVE AND FUNNY. I WANT TO LAUGH.
+++++
{{$input}}
+++++
```

Note the special **`{{$input}}`** token, which is a variable that is automatically passed when invoking the function, commonly referred to as a "function parameter". 

We'll explore later how functions can accept multiple variables, as well as invoke other functions.


In the same folder you'll notice a second [config.json](../../skills/FunSkill/Joke/config.json) file. The file is optional, and is used to set some parameters for large language models like Temperature, TopP, Stop Sequences, etc.

```
{
  "schema": 1,
  "type": "completion",
  "description": "Generate a funny joke",
  "completion": {
    "max_tokens": 500,
    "temperature": 0.5,
    "top_p": 0.5
  }
}
```

Given a semantic function defined by these files, this is how to load and use a file based semantic function.

Load and configure the kernel, as usual, loading also the AI service settings defined in the [Setup notebook](00-getting-started.ipynb):

In [ ]:
!python -m pip install semantic-kernel==0.2.7.dev0

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion, OpenAITextCompletion

kernel = sk.Kernel()

useAzureOpenAI = True

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", AzureTextCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", OpenAITextCompletion("text-davinci-003", api_key, org_id))

Import the skill and all its functions:

In [ ]:
# note: using skills from the samples folder
skills_directory = "../../skills"

funFunctions = kernel.import_semantic_skill_from_directory(skills_directory, "FunSkill")

jokeFunction = funFunctions["Joke"]

How to use the skill functions, e.g. generate a joke about "*time travel to dinosaur age*":

In [ ]:
context = kernel.create_new_context()
context["style"] = "Shakespeare"

result = jokeFunction(input="time travel to dinosaur age")

print(result)

# You can also invoke functions asynchronously
# result = await jokeFunction.invoke_async("time travel to dinosaur age")
# print(result)

Great, now that you know how to load a skill from disk, let's show how you can [create and run a semantic function inline.](./03-semantic-function-inline.ipynb)

In [ ]:
writeFunctions = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")

rewriteFunction = writeFunctions["Rewrite"]

In [ ]:
context = kernel.create_new_context()
context["style"] = "Shakespeare"
target_text = """
I was walking along a street when I saw my friend. We stopped to say 'Hello' and catch up with recent events.
"""
result = rewriteFunction(target_text, context=context)
print(result)

In [ ]:
[x.name for x in rewriteFunction.parameters]

In [ ]:
chapterFunction = writeFunctions["NovelChapter"]

In [ ]:
[x.name for x in chapterFunction.parameters]

In [ ]:
result = chapterFunction("A man comes to a house", theme="A Victorian gothic novel of unrequited love and tragedy")
print(result)